Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [2]:
import pandas as pd
import numpy as np
from itertools import product
#from grader import Grader

# Read data

In [3]:
# sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')
sales = pd.read_csv('data/sales_train.csv')

In [34]:
sales.shape

(2935849, 6)

In [5]:
sales.item_cnt_day.mean()

1.242640885140891

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [6]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

In [7]:
grid[0]

array([[   59, 22154,     0],
       [   59,  2552,     0],
       [   59,  2554,     0],
       ...,
       [   45,   628,     0],
       [   45,   631,     0],
       [   45,   621,     0]])

In [8]:
grid[1]

array([[   50,  3880,     1],
       [   50,  4128,     1],
       [   50,  4124,     1],
       ...,
       [   28, 12885,     1],
       [   28, 12791,     1],
       [   28, 13433,     1]])

In [9]:
len(grid), len(sales['date_block_num'].unique())

(34, 34)

In [10]:
#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [11]:
grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [16]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [24]:
#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'}).rename(columns={'item_cnt_day':'target'})

In [25]:
#fix column names
# gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [26]:
all_data.target.mean()

0.3342730567123426

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [27]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


#### Method 2

In [28]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [50]:
# grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [29]:
from sklearn.model_selection import KFold

In [30]:
cv = KFold(n_splits=5, shuffle=False, random_state=42)

C:\Users\luisr\Anaconda3\envs\data\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [31]:
mapped_folds = []
for comp_idx, fold_idx in cv.split(all_data):
    comp = all_data.iloc[comp_idx,:]
    fold = all_data.iloc[fold_idx,:]
    item_id_target_mean = comp.groupby('item_id').target.mean()
    mapped_folds.append(fold['item_id'].map(item_id_target_mean).fillna(0.3343))

In [32]:
encoded_feature = pd.concat(mapped_folds)

In [33]:
all_data.item_target_enc = encoded_feature

In [34]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.334300
141495,0,27,0,0.0,0.048523
144968,0,28,0,0.0,0.142424
142661,0,29,0,0.0,0.030303
138947,0,32,0,6.0,0.894020


In [35]:
encoded_feature = all_data['item_target_enc'].values

In [37]:
# YOUR CODE GOES HERE

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(np.round(corr,4))
# grader.submit_tag('KFold_scheme', pd.round(corr,4))

0.41645907127988024
0.4165


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [54]:
all_data

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.334300
141495,0,27,0,0.0,0.048523
144968,0,28,0,0.0,0.142424
142661,0,29,0,0.0,0.030303
138947,0,32,0,6.0,0.894020
...,...,...,...,...,...
10768834,59,22162,33,0.0,0.020000
10769024,59,22163,33,0.0,0.334300
10769690,59,22164,33,0.0,0.334300
10771216,59,22166,33,0.0,0.349227


In [55]:
id_sum_count = pd.concat([all_data.item_id, all_data.groupby('item_id')['target'].transform('sum').rename('sum'), all_data.groupby('item_id')['target'].transform('count').rename('count'), all_data.target], axis=1)

In [56]:
id_sum_count

,item_id,sum,count,target
139255,19,1.0,45,0.0
141495,27,42.0,739,0.0
144968,28,84.0,595,0.0
142661,29,12.0,321,0.0
138947,32,2092.0,1586,6.0
...,...,...,...,...
10768834,22162,699.0,449,0.0
10769024,22163,75.0,129,0.0
10769690,22164,493.0,399,0.0
10771216,22166,348.0,1176,0.0


In [61]:
all_data.item_target_enc = (id_sum_count['sum'] - id_sum_count.target)/(id_sum_count['count'] - 1)
encoded_feature = all_data['item_target_enc'].values

In [62]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(np.round(corr,4))
#grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129305


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [69]:
all_data['item_mean'] = all_data.groupby('item_id')['target'].transform('mean')
all_data['item_count'] = all_data.groupby('item_id')['target'].transform('count')

In [72]:
alpha = 100
globalmean = 0.3343

In [76]:
all_data.item_target_enc = (all_data.item_mean * all_data.item_count + alpha * globalmean)/(alpha + all_data.item_count)
encoded_feature = all_data['item_target_enc'].values

In [77]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(np.round(corr,4))
#grader.submit_tag('Smoothing_scheme', corr)

0.4818198797097282
0.4818


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [78]:
all_data['item_target_cumsum'] = all_data.groupby('item_id')['target'].transform('cumsum')
# all_data['item_cumcount'] = all_data.groupby('item_id')['target'].transform('cumcount')

In [79]:
all_data

,shop_id,item_id,date_block_num,target,item_target_enc,item_count,item_mean,item_target_cumsum,item_cumcount
139255,0,19,0,0.0,0.237448,45,0.022222,0.0,0.0
141495,0,27,0,0.0,0.089905,739,0.056834,0.0,0.0
144968,0,28,0,0.0,0.168964,595,0.141176,0.0,0.0
142661,0,29,0,0.0,0.107910,321,0.037383,0.0,0.0
138947,0,32,0,6.0,1.260635,1586,1.319042,6.0,0.0
...,...,...,...,...,...,...,...,...,...
10768834,59,22162,33,0.0,1.334117,449,1.556793,699.0,2.0
10769024,59,22163,33,0.0,0.473493,129,0.581395,75.0,2.0
10769690,59,22164,33,0.0,1.054970,399,1.235589,493.0,2.0
10771216,59,22166,33,0.0,0.298926,1176,0.295918,348.0,2.0


In [88]:
all_data['item_cumcount'] = all_data.groupby('item_id').cumcount()

In [90]:
all_data.query('item_id == 22167')

,shop_id,item_id,date_block_num,target,item_target_enc,item_count,item_mean,item_target_cumsum,item_cumcount
3179428,2,22167,8,0.0,1.02455,1221,1.081081,0.0,0
3163676,3,22167,8,0.0,1.02455,1221,1.081081,0.0,1
3171552,4,22167,8,0.0,1.02455,1221,1.081081,0.0,2
3187304,5,22167,8,0.0,1.02455,1221,1.081081,0.0,3
3147924,6,22167,8,0.0,1.02455,1221,1.081081,0.0,4
...,...,...,...,...,...,...,...,...,...
10759685,55,22167,33,0.0,1.02455,1221,1.081081,1320.0,1216
10748859,56,22167,33,0.0,1.02455,1221,1.081081,1320.0,1217
10754272,57,22167,33,0.0,1.02455,1221,1.081081,1320.0,1218
10765098,58,22167,33,0.0,1.02455,1221,1.081081,1320.0,1219


In [95]:
all_data.item_target_enc = ((all_data.item_target_cumsum - all_data.target)/all_data.item_cumcount).fillna(globalmean)
encoded_feature = all_data['item_target_enc'].values

In [96]:
encoded_feature

array([0.3343    , 0.3343    , 0.3343    , ..., 1.23869347, 0.29617021,
       1.08196721])

In [97]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
print(np.round(corr,4))
# grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245211081701
0.5025


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = # EMAIL HERE
STUDENT_TOKEN = # TOKEN HERE
grader.status()

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)